In [24]:
# import required libs
import numpy as np
import pandas as pd

import warnings
import math
import requests

warnings.filterwarnings('ignore')

In [25]:
# fetching a data table from the Fundamentus site and creating a dataFrame
url = 'https://www.fundamentus.com.br/resultado.php'

header = {
    "User-Agent":"Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3"
}

# get HTML from URL
req = requests.get(url, headers=header)
# read the HTML preparing the data
dataFrame = pd.read_html(req.text, decimal=',', thousands='.')[0]

print("[DF] " + dataFrame['Papel'])

0       [DF] CSTB4
1       [DF] CFLU4
2       [DF] POPR4
3       [DF] MNSA3
4       [DF] CLAN3
          ...     
982    [DF] UBBR11
983     [DF] GUAR4
984     [DF] UBBR3
985     [DF] MLFT4
986    [DF] BBTG12
Name: Papel, Length: 987, dtype: object


In [26]:
# dataFrame data handling
for column in ['Div.Yield', 'Mrg Ebit', 'Mrg. Líq.', 'ROIC', 'ROE', 'Cresc. Rec.5a']:
    dataFrame[column] = dataFrame[column].str.replace('.', '')
    dataFrame[column] = dataFrame[column].str.replace(',', '.')
    dataFrame[column] = dataFrame[column].str.rstrip('%').astype('float') / 100

# filtering only companies with stock market liquidity
dataFrame = dataFrame[ (dataFrame['Liq.2meses'] > 1000000) & (dataFrame['P/L'] > 0 ) ]

In [31]:
# creating ranking through Joel Greenblatt's magic formulas
ranking = pd.DataFrame()
# ranking of 100 positions
ranking['pos'] = range(1,101)
# ranks the top 100 stocks in terms and EV/EBIT
ranking['EV/EBIT'] = dataFrame[ dataFrame['EV/EBIT'] > 0 ].sort_values(by=['EV/EBIT'])['Papel'][:100].values
# ranks the best 100 shares in terms and P/L
ranking[ 'P/L'] = dataFrame.sort_values(by=['P/L'])['Papel'][:100].values
# ranks the top 100 stocks in terms of ROE
ranking['ROE'] = dataFrame.sort_values(by=['ROE'], ascending=False)['Papel'][:100].values
# ranks the top 100 stocks in terms of ROIC
ranking['ROIC'] = dataFrame.sort_values(by=['ROIC'], ascending=False)['Papel'][:100].values
# print the ranking
print(ranking)

    pos EV/EBIT     P/L    ROE    ROIC
0     1   PSSA3   MRFG3  BEEF3   UNIP6
1     2   GOAU3   LUPA3  BBSE3   UNIP3
2     3   GOAU4   PETR4  MRFG3   PSSA3
3     4   PETR4   PETR3  SUZB3   KEPL3
4     5   PETR3   SUZB3  KEPL3   FESA4
..  ...     ...     ...    ...     ...
95   96   EGIE3   BPAN4  GMAT3   USIM3
96   97   SBFG3  SANB11  TRPL4   COCE5
97   98   AMBP3   BLAU3  PORT3   DXCO3
98   99   TIMS3   SMTO3  JSLG3  ENGI11
99  100   TAEE3   USIM5  BOAS3   CSMG3

[100 rows x 5 columns]


In [32]:
# handling ranking dataFrame
a = ranking.pivot_table(columns='EV/EBIT')
b = ranking.pivot_table(columns='ROIC')
c = ranking.pivot_table(columns='P/L')
d = ranking.pivot_table(columns='ROE')
table = pd.concat([a,b,c,d])

print(table)

     ALUP11  AMBP3  ASAI3  AURA33  BEEF3  BLAU3  BMOB3  BRIT3  CBAV3  CCRO3  \
pos    52.0   98.0   62.0      56     42     72   69.0   92.0     28     39   
pos    91.0    NaN   45.0      35     28     15   62.0    NaN     93     25   
pos    86.0    NaN    NaN      76     75     98    NaN    NaN     16     58   
pos     NaN    NaN   32.0      53      1     64    NaN    NaN     82     21   

     ...  POMO3  POMO4  SANB11  SANB3  SMTO3  TRIS3  BBSE3  BPAC11  CXSE3  \
pos  ...    NaN    NaN     NaN    NaN    NaN    NaN    NaN     NaN    NaN   
pos  ...    NaN    NaN     NaN    NaN    NaN    NaN    NaN     NaN    NaN   
pos  ...   47.0   67.0    97.0   88.0   99.0   89.0    NaN     NaN    NaN   
pos  ...   94.0   95.0     NaN    NaN   67.0    NaN    2.0    73.0   39.0   

     PORT3  
pos    NaN  
pos    NaN  
pos    NaN  
pos   98.0  

[4 rows x 156 columns]


In [33]:
# excluding assets that have any missing data and leaving only the first 10 for use
rank = table.dropna(axis=1).sum()
# printing rank
rank = rank.sort_values()[:20]
print(rank)

PETR4     25
PETR3     27
KEPL3     31
UNIP3     50
TASA4     54
UNIP6     56
VALE3     68
FESA4     70
GOAU4     78
GOAU3     78
GGBR3     85
GGBR4     90
SUZB3     91
RECV3    107
WIZC3    118
RANI3    132
PTBL3    141
CCRO3    143
BEEF3    146
LEVE3    148
dtype: int64


In [35]:
## STARTING BENJAMIN GRAHAM'S FAIR PRICE FILTER
# setting dataFrame coming from the site by the name of the asset

dataFrame2 = dataFrame.set_index('Papel')

# creation and execution of the Graham filter
indicate = []
for i in range(10):
    pl = dataFrame2.loc[rank.index[i], 'P/L']
    pvp = dataFrame2.loc[rank.index[i], 'P/VP']
    cot = dataFrame2.loc[rank.index[i], 'Cotação']
    lpa = cot / pl
    vpa = cot / pvp
    grahamValue = math.sqrt(22.5 * lpa * vpa)
    if (grahamValue > cot):
        indicate.append(rank.index[i])

print('Lista de ativos recomendados seguindo a Formula Mágica (Joel Greenblatt) e filtrados pelo preço justo de Graham:')
print(indicate)

Lista de ativos recomendados seguindo a Formula Magica e filtrados pelo preço justo de Graham:
['PETR4', 'PETR3', 'KEPL3', 'UNIP3', 'TASA4', 'UNIP6', 'VALE3', 'FESA4', 'GOAU4', 'GOAU3']
